In [6]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc, html, Dash, State
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
from dash.exceptions import PreventUpdate

mapbox_token = 'pk.eyJ1IjoiYW5kcmVhc29zdGVkIiwiYSI6ImNsbmJxMjFndDA4dm8ybXJrMzhia2NqdnoifQ.fXDNIJ1LelhA1ypNiaJE9w'

Data wrangling

In [7]:
df = pd.read_csv('Data/homicide_data.csv')
citydata= pd.read_csv('Data/us-cities-top-1k.csv')
statedata=pd.read_csv('Data/states.csv')
citydata=pd.merge(citydata,statedata,on="State",how="left")
df['victim_full_name'] = df['victim_first'] +  ' ' + df['victim_last']

df = df.sort_values('city')

df['reported_date'] = pd.to_datetime(df['reported_date'], format='%Y%m%d')
# Extract month from date and convert to string
df['month'] = df['reported_date'].dt.strftime('%Y-%m')
#avg_city_coords = citydata.groupby('city').agg({'lat': 'mean', 'lon': 'mean'}).reset_index()\
#               .rename(columns={"lat": "avg_lat", "lon": "avg_lon"})

df = pd.merge(df, citydata.rename(columns={"lat": "avg_lat", "lon": "avg_lon","Abbreviation":"state","State":"Long_state"}), on = ['city',"state"], how = 'left')
df['No_homocides_norm_pr1000']= (df.groupby('city')['city'].transform('count'))/df["Population"]
df['No_homocides_norm_pr1000']=df['No_homocides_norm_pr1000']*1000
avg_city_coords=df[['city','avg_lat','avg_lon','No_homocides_norm_pr1000','Population','state']]
avg_city_coords=avg_city_coords.drop_duplicates().reset_index()
avg_city_coords=avg_city_coords.drop(49)

In [8]:
obs = df.groupby(['disposition', 'city']).size().reset_index(name='n_obs')

# Pivot the DataFrame to have 'disposition' as columns and 'city' as index
obs_pivot = obs.pivot(index='city', columns='disposition', values='n_obs')

# Calculate the proportion of 'Open/No arrest' over the sum of all dispositions
obs_pivot['Open_prop'] = np.round(obs_pivot['Open/No arrest'] / obs_pivot.sum(axis=1),2)

# Reset index if you want to have 'city' as a regular column
obs_pivot = obs_pivot.reset_index()

In [9]:
avg_city_coords=pd.merge(avg_city_coords,obs_pivot[["city","Open_prop"]],on="city",how="left")

Dash app

In [10]:
app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP])

cityoptions = [{'label': 'Select all', 'value': 'all cities'}]+[{'label': city, 'value': city} for city in sorted(df['city'].unique())]
raceoptions = [{'label': 'Select all', 'value': 'all'}]+[{'label': r, 'value': r} for r in sorted(df['victim_race'].unique())]
sexoptions = [{'label': 'Select all', 'value': 'all'}]+[{'label': s, 'value': s} for s in sorted(df['victim_sex'].unique())]

# Define layout
app.layout = html.Div([
    html.H4(id='title-output',style={"margin-bottom":"5px","margin-top":"1px","font-size":"24px","text-align":"center"}),
    html.Hr(style={"margin":"0px","marginBottom":"4px"}),
    dcc.Dropdown(
        id='city_dropdown',
        options = cityoptions,
        value = 'all cities', #default
        #value = None,
        #placeholder = "Select city",
        clearable=False,
        style={"width": "200px", "display": "inline-block","margin-top":"0"}
    ),
        dcc.Dropdown(
        id='race_dropdown',
        options = raceoptions,
        value = 'all', #default
        #value = None,
        #placeholder = "Select race",
        clearable=False,
        style={"width": "200px", "display": "inline-block","margin-top":"0"}
    ),
        dcc.Dropdown(
        id='sex_dropdown',
        options = sexoptions,
        value = 'all', #default
        #value = None,
        #placeholder = "Select gender",
        clearable=False,
        style={"width": "200px", "display": "inline-block","margin-top":"0"}
    ),
    dbc.Card([dcc.Graph(id = 'murder-map',style={"height":"48vh"})]
             ,style={"width": "60%","height":"48vh"}),
    dbc.Card([dcc.Graph(id = 'disp-colchart',style={"height":"37vh"})],style={"width": "35%","height":"37vh","marginTop":"10px"}),
    dcc.Graph(id = 'arrested-linechart'),
dcc.Store(id='selected-city', data = 'all cities')
],
style={"marginLeft":"50px"})

#unique_dispositions = df["disposition"].unique()
#custom_order=unique_dispositions
def make_colchart(df):
    
    # Get value counts and reset index
    value_counts = df["disposition"].value_counts().reset_index()
    
    fig = go.Figure(data=go.Bar(x=value_counts['disposition'], 
                                y=value_counts['count'],
                                marker_color="Gold"))
    fig.update_layout(dragmode='zoom')
    fig.update_layout(margin={"r": 5, "t": 5, "l": 5, "b": 5}) 
    fig.update_layout(autosize = True)
    return fig


def make_linechart(df):
    if len(df) == 0:
        fig = px.scatter(title='No data available')
        return fig
    else:
        grouped = df.groupby(['month', 'disposition']).size().unstack(fill_value=0)

        # Calculate the proportion of closed cases by arrest
        grouped['closed_arrested'] = grouped['Closed by arrest'] / grouped.sum(axis=1)

        # Calculate the rolling average
        window = 6  # You can adjust the window size
        grouped['smoothed'] = grouped['closed_arrested'].rolling(window=window).mean()

        # Create a line chart
        fig = px.line(grouped, 
                    x=grouped.index, 
                    y='closed_arrested', 
                    title='Proportion of Closed Cases by Arrest Over Time')

        # Add smoothed line
        fig.add_scatter(x=grouped.index, y=grouped['smoothed'], mode='lines', name='Smoothed', line=dict(color='red'))

        return fig

def make_map2(df, city):
    jitter_factor = 0.0005  # Adjust the jitter factor as needed
    df['lat'] = df['lat'] + np.random.uniform(-jitter_factor, jitter_factor, len(df))
    df['lon'] = df['lon'] + np.random.uniform(-jitter_factor, jitter_factor, len(df))
    
    fig = px.scatter_mapbox(
        df if city != 'all cities' else avg_city_coords,
        lat="lat" if city != 'all cities' else 'avg_lat',
        lon="lon" if city != 'all cities' else 'avg_lon',
        hover_name="victim_full_name" if city != 'all cities' else 'city',
        hover_data=["victim_race", "victim_age", "victim_sex", "reported_date"] if city != 'all cities' else None,
        color = "disposition"  if city != 'all cities' else "Open_prop",
        color_continuous_scale="YlOrRd" if city=='all cities' else None, #"Rdbu_r" 
        labels={"Open_prop": "% Open cases"},
        size='No_homocides_norm_pr1000' if city=='all cities' else None,
        center={"lat": 39.8283-2, "lon": -98.5795} if city=='all cities' else None,
        range_color=[min(avg_city_coords["Open_prop"])-0.15,max(avg_city_coords["Open_prop"])] if city=='all cities' else None,
        color_continuous_midpoint=0.5 if city=='all cities' else None
    )
    #title = f'Murders in {city}' if city is not None else ''

    #if city == 'all cities':
    #    fig.update_traces(marker={'size': 10})
    fig.update_coloraxes(colorbar_tickformat='.0%')
    fig.update_layout(mapbox_style="light", 
                      mapbox_accesstoken=mapbox_token,
                      #title = title,
                      #mapbox_center={"lat": center_lat, "lon": center_lon},  
                      #mapbox_zoom = 10,
                      autosize = True
                      )
    
    fig.update_layout(mapbox_zoom = 9.5 if city != 'all cities' else 3.2)

    fig.update_traces(uirevision='persist')
    
    fig.update_layout(margin={"r": 0, "l": 15, "b": 5,"t":5})
    fig.update_layout(clickmode='event+select',
                      hovermode='closest')
    return fig

'''
def make_map(city):
    filtered_df = df[df['city'] == city] if city != 'all cities' else df.groupby('city').agg({'lat': 'mean', 'lon': 'mean'}).reset_index()


    fig = px.scatter_mapbox(
        filtered_df,
        lat="lat",
        lon="lon",
        hover_name="victim_full_name" if city != 'all cities' else 'city',
        hover_data=["victim_race", "victim_age", "victim_sex"] if city != 'all cities' else None,
        color = "disposition"  if city != 'all cities' else None,
        height=700
    )
    
    title = f'Murders in {city}' if city is not None else ''

    
    fig.update_layout(mapbox_style="light", 
                      mapbox_accesstoken=mapbox_token, 
                      title = title,
                      #mapbox_center={"lat": center_lat, "lon": center_lon},  
                      #mapbox_zoom = 10,
                      autosize = True
                      )
    
    fig.update_layout(mapbox_zoom = 9 if city != 'all cities' else 3.9)

    fig.update_traces(uirevision='persist')
    
    fig.update_layout(margin={"r": 0, "l": 0, "b": 0})
    fig.update_layout(clickmode='event+select',
                      hovermode='closest')
    return fig
'''

#helper function to filter data based on city, race, sex
def filter_df(selected_city, selected_race, selected_sex, df = df):
    all_cities_filter = selected_city == 'all cities'
    all_race_filter = selected_race == 'all'
    all_sex_filter = selected_sex == 'all'

    if all_cities_filter and all_race_filter and all_sex_filter:
        # Handle the case when all filters are 'all'
        filtered_df = df  # Include all data
    else:
        city_filter = df['city'] == selected_city if not all_cities_filter else True
        race_filter = df['victim_race'] == selected_race if not all_race_filter else True
        sex_filter = df['victim_sex'] == selected_sex if not all_sex_filter else True

        filtered_df = df[city_filter & race_filter & sex_filter]
    return filtered_df



@app.callback(
    [Output('murder-map', 'figure'),
    Output('disp-colchart', 'figure'),
    Output('arrested-linechart', 'figure')],
    
    [Input('city_dropdown', 'value'),
     Input('race_dropdown', 'value'),
     Input('sex_dropdown', 'value')
    ]
)

def update_figs_on_dropdowns(selected_city, selected_race, selected_sex):
    filtered_df = filter_df(selected_city, selected_race, selected_sex)

    fig_map = make_map2(filtered_df, selected_city)
    fig_col = make_colchart(filtered_df)
    fig_line = make_linechart(filtered_df)
    fig_map.update_layout(transition={'duration': 500, 'easing': 'cubic-in-out'})

    return fig_map, fig_col, fig_line


@app.callback(
    [Output('disp-colchart', 'figure', allow_duplicate=True),
     Output('arrested-linechart', 'figure', allow_duplicate = True)], 
    [Input('murder-map', 'selectedData')],
    [State('city_dropdown', 'value'), State('race_dropdown', 'value'), State('sex_dropdown', 'value')],
    prevent_initial_call = True
)
def update_colchart_on_map_selection(selected_data, selected_city, selected_race, selected_sex):
    if selected_data:
        # Extract the selected points from the map
        selected_points = selected_data['points']

        # Get the victim_full_name(s) from the selected points
        selected_names = [point['hovertext'] for point in selected_points]

        # Filter the DataFrame based on selected names, city, race, and sex
        filtered_df = df[df['victim_full_name'].isin(selected_names)]
        filtered_df = filter_df(selected_city, selected_race, selected_sex, filtered_df)

        # Generate the column chart using the filtered DataFrame
        fig_col = make_colchart(filtered_df)
        fig_line = make_linechart(filtered_df)
        return fig_col, fig_line
    else:
        fig_col = make_colchart(filter_df(selected_city, selected_race, selected_sex))
        fig_line = make_linechart(filter_df(selected_city, selected_race, selected_sex))
        return fig_col, fig_line
    



@app.callback(
    Output('city_dropdown', 'value'),
    Input('selected-city', 'data'),
    prevent_initial_call = True
)
def update_city_dropdown(selected_city):
    return selected_city

@app.callback(
    Output('selected-city', 'data'),
    [Input('murder-map', 'clickData')],
    [State('city_dropdown', 'value')]
)
def update_on_click(click_data, selected_city):
    if selected_city == 'all cities':
        if click_data:
            selected_city = click_data['points'][0]['hovertext']
            return selected_city
    raise PreventUpdate

@app.callback(
    Output('title-output', 'children'),
    [Input('city_dropdown', 'value')]
)
def update_title(city):
    city_list = [city for city in df['city'].unique()]
    if city in city_list:
        return f'Murders in {city}' 
    else:
        return 'Murders in 50 large cities in the US'
    


if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode = "tab", port = 8050)


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

Testing things

In [ ]:
citymerge=pd.merge(avg_city_coords,df[["city","victim_race","victim_sex"]],on='city',how="left")
citymerge=citymerge[['city','avg_lat','avg_lon','No_homocides_norm_pr1000','Population','state',"victim_sex","victim_race"]]
filtered_city = filter_df("all cities", "Black", "Male",citymerge)
filtered_city[['city','avg_lat','avg_lon','No_homocides_norm_pr1000','Population','state']].drop_duplicates().reset_index()

,index,city,avg_lat,avg_lon,No_homocides_norm_pr1000,Population,state
0,25,Albuquerque,35.085334,-106.605553,0.679251,556495.0,NM
1,379,Atlanta,33.748995,-84.387982,2.172646,447841.0,GA
2,1351,Baltimore,39.290385,-76.612189,4.544256,622104.0,MD
3,4178,Baton Rouge,30.458283,-91.140320,1.848090,229426.0,LA
4,4602,Birmingham,33.520661,-86.802490,3.771575,212113.0,AL
5,5402,Boston,42.360082,-71.058880,0.950514,645966.0,MA
6,6018,Buffalo,42.886447,-78.878369,2.011901,258959.0,NY
7,6537,Charlotte,35.227087,-80.843127,0.866481,792862.0,NC
8,7224,Chicago,41.878114,-87.629798,2.035838,2718782.0,IL
9,12759,Cincinnati,39.103118,-84.512020,2.332640,297517.0,OH


In [ ]:
filtered_df = filter_df('Birmingham', 'Hispanic', 'all')
make_linechart(filtered_df)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'month=%{x}<br>closed_arrested=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array(['2007-06', '2007-09', '2007-12', '2008-06', '2008-09', '2008-11',
                          '2009-01', '2009-02', '2009-04', '2010-03', '2010-04', '2010-12',
                          '2011-09', '2014-09', '2015-09', '2015-10', '2015-11', '2016-07',
                          '2016-09', '2016-10'], dtype=object),
              'xaxis': 'x',
              'y': array([0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
                          0., 1.]),
              'yaxis': 'y'},
             {'line': {'color': 'red'},
              'mode': 'lines',
              'name': 'Smoothed',
              'type': 'scatter',
              'x': array(['2007-06', '2007-09', '2007-12', '2008-06', '2008-09', '2008-11',
                          '2009-01', '2009-02', '2009-04', '2010-03', '2010-04', '2010-12',
                          '2011-09', '2014-09', '2015-09', '2015-10', '2015-11', '2016-07',
                          '2016-09', '2016-10'], dtype=object),
              'y': array([       nan,        nan,        nan,        nan,        nan, 0.16666667,
                          0.16666667, 0.33333333, 0.5       , 0.66666667, 0.66666667, 0.83333333,
                          1.        , 0.83333333, 0.66666667, 0.5       , 0.33333333, 0.33333333,
                          0.16666667, 0.33333333])}],
    'layout': {'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Proportion of Closed Cases by Arrest Over Time'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'month'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'closed_arrested'}}}
})